In [1]:
import pandas as pd
import ta
from resources import perform_grid_search, perform_backtest
import warnings
warnings.filterwarnings('ignore')


In [2]:
def get_training(df, row):
    return df[(df['timestamp'] >= row['4_days_before_start']) & (df['timestamp'] < row['start'])]

In [3]:
def get_test(df, row):
    return df[(df['timestamp'] >= row['start']) & (df['timestamp'] < row['end'])]

In [4]:
groups = pd.read_csv('groups.csv')

In [5]:
groups['start'] = pd.to_datetime(groups['start'])
groups['end'] = pd.to_datetime(groups['end'])

In [6]:
groups['4_days_before_start'] = groups['start'] - pd.Timedelta(days=4)

In [7]:
df = pd.read_csv('features.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'percentage_large', 'buy_percentage_large']]

In [8]:
df = df.fillna(method='ffill').fillna(method='bfill')

In [9]:
groups

,start,end,name,4_days_before_start
0,2020-02-28,2020-03-23,corona_dump,2020-02-24
1,2020-03-23,2020-04-24,corona_recovery,2020-03-19
2,2020-04-29,2020-05-15,further_recovery,2020-04-25
3,2020-05-15,2020-06-04,9k_accumulation,2020-05-11
4,2020-06-06,2020-06-28,further_9k_accumulation,2020-06-02
5,2020-07-01,2020-07-24,final_9k_accumulation,2020-06-27
6,2020-07-24,2020-08-14,initial_takeoff,2020-07-20
7,2020-08-14,2020-09-03,12k_dump,2020-08-10
8,2020-09-03,2020-09-16,10k_accumulation,2020-08-30
9,2020-09-16,2020-10-11,10k_accumulation_2,2020-09-12


In [10]:
#NUMBER 0#

In [11]:
new_group = pd.DataFrame()

for idx, row in groups.iterrows():
    row['4_days_before_start_price'] = df[df['timestamp'] == row['4_days_before_start']].iloc[0]['close']
    row['start_price'] = df[df['timestamp'] == row['start']].iloc[0]['close']
    row['creation_change'] = (row['start_price'] - row['4_days_before_start_price'])/row['4_days_before_start_price'] * 100
    
    row['end_price'] = df[df['timestamp'] == row['end']].iloc[0]['close']
    row['hodl_return'] = (row['end_price'] - row['start_price'])/row['start_price'] * 100
    
    new_group = new_group.append(row, ignore_index=True)

In [12]:
new_group = new_group[['name', '4_days_before_start', '4_days_before_start_price', 'start', 'start_price', 'creation_change', 'end' ,'end_price', 'hodl_return']]

In [13]:
def long_or_short(x):
    if x < 0:
        return "SHORT"
    if x > 0:
        return "LONG"

In [14]:
new_group['trade_direction'] = new_group['creation_change'].apply(long_or_short)

In [15]:
new_group

,name,4_days_before_start,4_days_before_start_price,start,start_price,creation_change,end,end_price,hodl_return,trade_direction
0,corona_dump,2020-02-24,9970.089731,2020-02-28,8801.267383,-11.723288,2020-03-23,5867.999939,-33.327785,SHORT
1,corona_recovery,2020-03-19,5448.997503,2020-03-23,5867.999939,7.689533,2020-04-24,7506.943923,27.930198,LONG
2,further_recovery,2020-04-25,7491.197843,2020-04-29,7762.769756,3.625213,2020-05-15,9773.260360,25.899140,LONG
3,9k_accumulation,2020-05-11,8756.567426,2020-05-15,9773.260360,11.610633,2020-06-04,9645.992090,-1.302209,LONG
4,further_9k_accumulation,2020-06-02,10214.504597,2020-06-06,9573.956284,-6.270968,2020-06-28,8992.805755,-6.070119,SHORT
5,final_9k_accumulation,2020-06-27,9131.586157,2020-07-01,9119.094887,-0.136792,2020-07-24,9607.996755,5.361298,SHORT
6,initial_takeoff,2020-07-20,9222.539887,2020-07-24,9607.996755,4.179509,2020-08-14,11837.120091,23.200709,LONG
7,12k_dump,2020-08-10,11720.581341,2020-08-14,11837.120091,0.994309,2020-09-03,11394.712660,-3.737458,LONG
8,10k_accumulation,2020-08-30,11488.970588,2020-09-03,11394.712660,-0.820421,2020-09-16,10745.756119,-5.695243,SHORT
9,10k_accumulation_2,2020-09-12,10380.983242,2020-09-16,10745.756119,3.513857,2020-10-11,11309.658448,5.247675,LONG


In [14]:
#create long term trends of buy, sell

In [20]:
def get_strategy_return(ser):
    if ser['trade_direction'] == 'LONG':
        return ser['hodl_return']
    elif ser['trade_direction'] == 'SHORT':
        return ser['hodl_return'] * -1

In [22]:
new_group['strategy_return'] = new_group.apply(get_strategy_return, axis=1)

In [40]:
new_group = new_group.round(2)

In [41]:
new_group.to_csv('test_1.csv', index=None)

In [31]:
starting = 1000
vals = 1+new_group['hodl_return'].values/100

for x in vals:
    starting = starting * x

In [33]:
starting

1870.6171326122212

In [34]:
starting = 1000
vals = 1+new_group['strategy_return'].values/100

for x in vals:
    starting = starting * x

In [35]:
starting

4252.648306901988